# Advent of Code 2019, Dyalog APL edition

To see a correct render of this notebook, check it out on [nbviewer](https://nbviewer.jupyter.org/github/xpqz/AoCDyalog/blob/master/Advent%20of%20Code%202019%20Dyalog%20APL.ipynb).

Annotated solutions in Dyalog APL. Why? A language that doesn't affect the way you think about programming is not worth knowing.

Note that part of the charm of AoC is that every user (or at least groups of users) gets their own unique data set. Some of the solutions below exploit quirks in my particular data set, and so may conceivably not work for the general case.

In [64]:
⍝ Helper functions and common settings
⎕FR ⎕PP ⎕IO←1287 34 0
assert←{⍺←'assertion failure' ⋄ 0∊⍵:⍺ ⎕signal 8 ⋄ shy←0}
lines←{⊃⎕NGET ⍵ 1}
line←{⊃lines ⍵}
sorted←{⍵[⍋⍵]}
sortbycol←⊢⌷⍨∘⊂∘⍒⌷⍤1
pairs←{↓(2÷⍨≢⍵) 2⍴⍵}
bin←{(32⍴2)⊤⍵}
dec←2∘⊥
and←{(bin ⍺)∧bin ⍵}
or←{(bin ⍺)∨bin ⍵}
rs←⊢∘-∘≢↑↓⍨∘-⍨
range←⊣+∘⍳-⍨
group←↓⊃¨,∘⊂⌸⊢/¨
replace←{⍺⍺(⍵⍵⌷⍨∘⊂⍳)@(∊∘⍺⍺)⍵}
cut←⊢⊂⍨≢⍤⊢⍴⊣↑≢

In [65]:
⍝ Some visualisation help, please
]box on -style=max -trains=tree -fns=on
]rows on

┌→─────────────────────────────────────┐
│Was ON -style=max -trains=tree -fns=on│
└──────────────────────────────────────┘

┌→─────┐
│Was ON│
└──────┘

### Day 1: The Tyranny of the Rocket Equation
https://adventofcode.com/2019/day/1

In [7]:
DAY01←⍎¨⊃⎕NGET'data/2019/01.txt'1
Fuel←{0⌈¯2+⌊⍵÷3}

In [9]:
⊢part1←+/Fuel DAY01
assert 3328306=part1

3328306

In [10]:
⊢part2←+/{⍵=0:0⋄(Fuel ⍵)+∇Fuel ⍵}¨DAY01
assert 4989588=part2

4989588

### Day 2: 1202 Program Alarm
https://adventofcode.com/2019/day/2

The first of MANY intcode tasks.

In [15]:
DAY02←⍎line'data/2019/02.txt'

In [22]:
]dinput
IntcodeV1←{
    ⍺←0                                          ⍝ Intcode interpreter. 
    (op p1 p2 p3)←4↑⍺↓⍵                          ⍝ Skip anything before ip (⍺) and take 4 cells
    op∊1 2:(⍺+4)∇((op-1)⌷⍵[p1](+,×)⍵[p2])@p3 ⊢ ⍵ ⍝ Addition and multiplication
    op=99:⍵[0]                                   ⍝ Exit
}

In [23]:
DAY02[1 2] ← 12 2
⊢part1←IntcodeV1 DAY02
assert 10566835=part1

10566835

In [24]:
Day02p2←{19690720=IntcodeV1 ((⊃1↑⍵)@1 2)⊢⍺:⊃1↑⍵⋄⍺∇1↓⍵}
(noun verb)←DAY02 Day02p2 {,⍳⍵ ⍵} 100 
⊢part2←verb+100×noun
assert 2347=part2

2347

### Day 3: Crossed Wires
https://adventofcode.com/2019/day/3

In [12]:
DAY03←1(↑,∘⍎↓)¨⎕CSV'data/2019/03.txt' ⍝ Convert each string to a letter and a number.
OFFSETS←4 2⍴0 1 0 ¯1 1 0 ¯1 0

In [13]:
]dinput
Follow←{
    ⍺←0 0
    0=≢⍵:⍺
    (dir steps)←⊃⍵
    seq←,⌿OFFSETS['UDRL'⍳dir;]∘.×1+⍳steps
    origin←(≢seq)⍴⊂¯2↑∊⍺
    ⍺,∊(seq+origin)∇1↓⍵                  
}

In [14]:
(path1 path2)←pairs∘Follow¨↓DAY03

In [15]:
crossings←1↓∪path1∩path2 ⍝ Find intersections; drop the start point which is shared.

In [16]:
⊢part1←⌊/(+/|)¨crossings
assert 860=part1

860

In [17]:
⊢part2←⌊/+/(path1⍳crossings),⍪path2⍳crossings
assert 9238=part2

9238

### Day 4: Secure Container
https://adventofcode.com/2019/day/4

In [57]:
DAY04←236491 713787

In [59]:
⊢part1←+/{enc←(6⍴10)⊤⍵⋄(∧/2≤/enc)∧∨/2=/enc}¨(0⊃DAY04)range 1+1⊃DAY04
assert 1169=part1

1169

In [60]:
ngr←{2∊≢¨((1,2≠/⊢)⍵)⊂⍵} ⍝ contains pair not part of larger group

In [61]:
⊢part2←+/{enc←(6⍴10)⊤⍵⋄(∧/2≤/enc)∧ngr enc}¨(0⊃DAY04)range 1+1⊃DAY04
assert 757=part2

757

### Day 5: Sunny with a Chance of Asteroids
https://adventofcode.com/2019/day/5

Better get used to it ⍨.

In [30]:
DAY05←⊢⌿⍎¨⎕CSV'data/2019/05.txt'

In [35]:
]dinput
IntcodeV2←{ ⍝ Intcode interpreter, mk2. Call as: 0 IntcodeV2 code
    state←⍵
    ev←{⍺=0: state[⍵] ⋄ ⍵}              ⍝ Position or immediate mode
    op p1 p2 p3←4↑⍺↓⍵                   ⍝ Skip anything before and take 4 cells
    params←4 4 2 2 3 3 4 4 1            ⍝ Number of parameters by opcode
    ops←(1+⍳8),99                       ⍝ Valid opcodes
    m3 m2 m1 o2 o1←(5⍴10)⊤op            ⍝ Unpack the param modes
    op←10⊥o2 o1                         ⍝ Repack the opcode, to go from (say) 1001 to 1
    count←params[ops⍳op]                ⍝ Number of params
    parmod←m1 m2 m3,⍪p1 p2 p3           ⍝ Table combining modes and params
    d1 d2 d3←3↑ev/(¯1+count)↑parmod     ⍝ Pick relevant number of params, and apply modes
    ip←⍺+count                          ⍝ Advance ip by the width of current instr
    op∊1 2:ip∇((op-1)⌷d1(+,×)d2)@p3⊢⍵   ⍝ Addition and multiplicatin
    op=3:ip∇INPUT@p1⊢⍵                  ⍝ Input
    op=4:d1,ip∇⍵                        ⍝ Output
    op∊5 6:⍵∇⍨ip d2⌷⍨(op-5)⌷d1(≠,=)0    ⍝ Jumps 
    op∊7 8:ip∇((op-7)⌷d1(<,=)d2)@p3⊢⍵   ⍝ Comparison < or =
    op=99:⍬                             ⍝ Exit
}

In [36]:
INPUT←1
⊢part1←⊃¯1↑0 IntcodeV2 DAY05
assert 16209841=part1

16209841

In [39]:
INPUT← 5
⊢part2←⊃0 IntcodeV2 DAY05
assert 8834787=part2

8834787

### Day 6: Universal Orbit Map
https://adventofcode.com/2019/day/6

In [53]:
DAY06←↓⍉↑'\w+'⎕S'&'¨lines'data/2019/06.txt'
PARENTS←⊃⍳/⊖DAY06
Path←{3::⍵⋄⍵,∇⍵⊃PARENTS}

In [55]:
⊢part1←≢∊Path¨PARENTS
assert part1=292387

292387

In [63]:
⊢part2←{¯2+≢⍺(∪~∩)⍵}/Path¨(1⊃DAY06)⍳'YOU' 'SAN'
assert 433=part2

433

### Day 7: Amplification Circuit
https://adventofcode.com/2019/day/7

Already fed up.

This is a resumable Intcode iterpreter -- fed a state vector (code ip rb input output) it will execute until it's either terminated normally on opcode 99, or blocked on input.

Yes, it's a tradfn. Don't @ me. I wrote it so you won't have to, and we can all get back to our lives.

This question's instructions are somewhat misunderstandable. The suggestion is that there is some difference in the 'wiring' between parts 1 and 2; this is either not the case, or part 1 works equally well with the circular wiring suggested in part 2.

Also, part 2 states:

    Don't restart the Amplifier Controller Software on any amplifier during this process. 
    Each one should continue receiving and sending signals until it halts.
    All signals sent or received in this process will be between pairs of amplifiers except 
    the very first signal and the very last signal. To start the process, a 0 signal is sent 
    to amplifier A's input exactly once.
    
This actually means "exactly once (per phase setting)", rather than actually exactly once. So in practice, both parts are identical apart from the phases to try.

In [3]:
'pmat'⎕CY'dfns'
DAY07←⊢⌿⍎¨⎕CSV'data/2019/07.txt'

In [4]:
]dinput
r←IntcodeV3 argv;code;ip;rb;in;out;halt;ev;op;p1;p2;p3;params;ops;m3;m2;m1;o2;o1;count;parmod;d1;d2;d3
(code ip rb in out halt)←argv
:If halt
    r←argv
    :Return
:EndIf
ev←{⍺=0:⍵⊃code⋄⍵}                          ⍝ Position or immediate mode
:While 1
    (op p1 p2 p3)←4↑ip↓code                ⍝ Skip anything before and take 4 cells
    :If op=99
        r←code ip rb in out 1
        :Return
    :EndIf
                      
    params←4 4 2 2 3 3 4 4 1              ⍝ Number of parameters by opcode
    ops←99,⍨1+⍳8                          ⍝ Valid opcodes
    (m3 m2 m1 o2 o1)←op⊤⍨5⍴10             ⍝ Unpack the param modes
    op←10⊥o2 o1                           ⍝ Repack the opcode, to go from (say) 1001 to 1
    count←params[ops⍳op]                  ⍝ Number of params
    parmod←m1 m2 m3,⍪p1 p2 p3             ⍝ Table combining modes and params
    (d1 d2 d3)←3↑ev/(¯1+count)↑parmod     ⍝ Pick relevant number of params, and apply modes

    ip+←count                             ⍝ Advance ip by the width of current instr
    :If op∊1 2                            ⍝ Addition or multiplication
        code[p3]←(op-1)⌷d1(+,×)d2
    :ElseIf op=3                          ⍝ Input
        :If 0=≢in                         ⍝ Blocked on input
            r←code (ip-count) rb in out 0 ⍝ Reverse one instruction
            :Return
        :EndIf
        code[p1]←0⊃in
        in←1↓in
    :ElseIf op=4                          ⍝ Output
        out,←d1
    :ElseIf op∊5 6                        ⍝ Jumps
        ip←ip d2⌷⍨(op-5)⌷d1(≠,=)0
    :ElseIf op∊7 8                        ⍝ Comparisons
        code[p3]←(op-7)⌷d1(<,=)d2
    :Else
        assert 0 ⍝ Unknown opcode
    :End
:EndWhile

In [11]:
]dinput
RunPhase←{
    amps←⍵
    amps[;3],←⍺⋄amps←↑IntcodeV3¨↓amps
    amps[0;3],←0
    amps[0;]←IntcodeV3 0⌷amps

    RunAmps←{
        ampId←5|⍵+1
        amps[ampId;3],←0⊃⊖⊃amps[⍵;4]
        ⊢amps[ampId;]←IntcodeV3 ampId⌷amps
    }

    _←{RunAmps¨⍳5}⍣{amps[4;5]}⊢⍬
    amps
}

In [12]:
]dinput
Day07←{
    state←(↑5⍴⊂(⍺)0 0(,⍬)(,⍬)0)
    ⌈/{⊃¯1↑⊃(4 4)⌷⍵ RunPhase state}¨⍵
}

In [14]:
⊢part1←DAY07 Day07 ↓pmat 5
assert 262086=part1

262086

In [15]:
⊢part2←DAY07 Day07 {5 6 7 8 9[⍵]}¨↓pmat 5
assert 5371621=part2

5371621

### Day 8: Space Image Format
https://adventofcode.com/2019/day/8

Briefly back to non-intcode, and proper arrays!

In [192]:
DAY08←100 6 25⍴⍎¨line'data/2019/08.txt'

In [193]:
Z←⊃⍋{+/0=∊DAY08[⍵;;]}¨⍳100 ⍝ Pick layer with the fewest zeros

In [194]:
⊢part1←(+/1=∊DAY08[Z;;])×+/2=∊DAY08[Z;;]
assert 1935=part1

1935

Part 2: decode the image, with the value 2 being transparent. We can use the dyadic form of transpose to great effect here, to group the 100 values that make up each pixel into vectors.

In [201]:
img←{(⊃⍸2≠⍵)⊃⍵}¨↓2 0 1⍉DAY08 ⍝ Find first non-2 value at each point along the 100 layers
' *'[img]                    ⍝ Image reads CFLUL

┌→────────────────────────┐
↓ ** **** * * * * │
│* * * * * * * │
│* *** * * * * │
│* * * * * * │
│* * * * * * * │
│ ** * **** ** **** │
└─────────────────────────┘

### Day 9: Sensor Boost
https://adventofcode.com/2019/day/9

Yeah. The fun didn't last long, did it. Our new Intcode interpreter needs to support the 'relative base' operation, opcode 9, and also allow addressing an arbitrary distance beyond the given program. The intention here is that this is the final version of Intcode interpreter. 

I learnt a lot from [voidhawk42](https://github.com/voidhawk42/aoc2019apl/blob/master/p09.dyalog) on this.

In [307]:
DAY09←⊢⌿⍎¨⎕CSV'data/2019/09.txt'

Given the state of a running intcode program, exectute one instruction and return the new state.

In [24]:
]dinput
Step←{ 
    (code ip rb in out halt blocked)←⍵
    evalPar←{⍺=0:⍵⊃code⋄⍺=2:(⍵+rb)⊃code⋄⍵}
    applyRelBase←{⍺=2:⍵+rb⋄⍵}
    (op p1 p2 p3)←4↑ip↓code
    params←4 4 2 2 3 3 4 4 2 1
    ops←99,⍨1+⍳9
    (m3 m2 m1 o2 o1)←op⊤⍨5⍴10
    op←10⊥o2 o1
    count←params[ops⍳op]
    parmod←(¯1+count)↑m1 m2 m3,⍪p1 p2 p3
    read←3↑evalPar/parmod
    write←3↑applyRelBase/parmod
    ip+←count
    op∊1 2:code ip rb in out 0 0⊣code[2⊃write]←(op-1)⌷read[0](+,×)1⊃read
    (op=3)∧0=≢in:code (ip-count) rb in out 0 1
    op=3:code ip rb (1↓in) out 0 0⊣code[0⊃write]←0⊃in
    op=4:code ip rb in (out,0⊃read) 0 0
    op∊5 6:code (ip (1⊃read)⌷⍨(op-5)⌷read[0](≠,=)0) rb in out 0 0
    op∊7 8:code ip rb in out 0 0⊣code[2⊃write]←(op-7)⌷read[0](<,=)read[1]
    op=9:code ip (rb+0⊃read) in out 0 0
    op=99:code ip rb in out 1 0
    assert 0
}

Now we can run a program to completion using power ⍣. Note that we need to pre-allocate sufficient 'RAM' for the execution.

In [309]:
Intcode←{⍺←⍬⋄Step⍣{5⊃⍺}⊢(⍵,512⍴0)0 0(,⍺)(,⍬)0 0} ⍝ ⍺ is input vector, ⍵ is the program

In [310]:
⊢part1←⊃4⊃1 Intcode DAY09
assert 2171728567=part1

2171728567

In [311]:
⊢part2←⊃4⊃2 Intcode DAY09
assert 49815=part2

49815

### Day 10: Monitoring Station
https://adventofcode.com/2019/day/10

Trigonometry? For each asteroid, calculate angle to every other asteroid, and find the max count of unique angles.

In [182]:
DAY10←↑lines'data/2019/10.txt'
ASTR←⍸'#'=DAY10

In [183]:
Deg←{0=⍵:0⋄(180÷○∘÷)⍵}
ATan2←12○⊣+0J1×⊢
Angle←{v←360-360|180+Deg ATan2/⍵-⍺⋄v=360:0⋄v} ⍝ 0 and 360 pointing 'north' and increasing clockwise

In [184]:
GROUPS←(≢∪)¨↓ASTR∘.Angle ASTR
WINNER←⊃⍒GROUPS
⊢part1←WINNER⊃GROUPS
assert 334=part1

334

Part 2: complete vaporization by giant laser. Starting pointing up, eliminate the first asteroid visible, and repeat the process whilst rotating clockwise. Which is the 200th asteroid to be eliminated?

For this we need both angles and distances.

In [185]:
Dist←{.5*⍨+/2*⍨⍵-⍺}
Visibility←{(⍺ Angle ⍵)((⍺ Dist ⍵),⍵)}

In [186]:
VIZ←group sorted (⊃ASTR[WINNER])∘Visibility¨ASTR

The VIZ vector now holds groups consisting of angle and triplets of distance and coordinates, in ascending order. We need to find the 200th coordinate, by iterating over the angles, clockwise, picking the first non-seen item from each distance-ordered list. If we remix the coordinate parts of VIZ, we should get them in the order required, and then we can pick directly the coordinate at index 199.

In [187]:
ZAPPED←⊃0⌷↓⍉↑{{1↓⊃⍵}¨1⊃⍵}¨VIZ ⍝ Pick out only the coordinates, and remix/zip ↓⍉↑.

In [188]:
⊢part2←199⊃ZAPPED
assert part2≡19 11 ⍝ Submittable result is 1119, y+x×100

┌→────┐
│19 11│
└~────┘

### Day 11: Space Police
https://adventofcode.com/2019/day/11

And with that fun out of the way, normal Intcode service returns.

In [312]:
DAY11←⊢⌿⍎¨⎕CSV'data/2019/11.txt'
Turn←{⍺ ⍵⌷4 2⍴3 1 0 2 1 3 2 0}
Move←{⊃⍵⌷⍺+(¯1 0)(0 1)(1 0)(0 ¯1)}

In [313]:
]dinput
Day11←{
    ⍺←0 
    code←⍵
    panel←100 100⍴0
    panel[50;50]←⍺
    robot←{⍵⊣⍵.(state dir count pos)←((code,512⍴0)0 0(,⍬)(,⍬)0 0) 0 0 (50 50)}⎕NS''
    painted←100 100⍴0
    _←{ 
        robot.state[3]←⊂,robot.pos⌷panel             ⍝ Input: current tile colour
        robot.state[4]←⊂,⍬                           ⍝ Reset output buffer
        robot.state←Step⍣{(5⊃⍺)∨2=≢4⊃⍺}⊢robot.state  ⍝ Run robot until two outputs or halt
        5⊃robot.state:⍬                              ⍝ Halted
        (colour dir)←4⊃robot.state
        panel[⊂robot.pos]←colour                     ⍝ Paint our current tile
        painted[⊂robot.pos]+←colour=1
        robot.dir←robot.dir Turn dir                 ⍝ Turn left or right, as directed
        ⊢robot.pos←(⊂robot.pos) Move robot.dir       ⍝ Take one step in new dir
    }⍣{5⊃robot.state}⊢⍬
    (painted) (panel)
}

In [314]:
(painted _)←Day11 DAY11
⊢part1←+/0≠∊painted
assert 2088=part1

2088

In [315]:
(_ panel)←1 Day11 DAY11
(' *'[panel])[50+⍳6;51+⍳39] ⍝ Part 2: message reads URCAFLCP

┌→──────────────────────────────────────┐
↓* * *** ** ** **** * ** *** │
│* * * * * * * * * * * * * *│
│* * * * * * * *** * * * *│
│* * *** * **** * * * *** │
│* * * * * * * * * * * * * │
│ ** * * ** * * * **** ** * │
└───────────────────────────────────────┘

### Day 12: The N-Body Problem
https://adventofcode.com/2019/day/12

In [278]:
DAY12←(4 1 ¯15)(¯8 ¯10 1)(9 4 ¯5)(¯2 6 4) ⍝ Flipped from xyz to zyx format

In [279]:
Gravity←{+/⍵∘.(<->)⍵} ⍝ Return velocity change, a.k.a acceleration
Energy←×⍥(+/∘|)
ApplyForces←{(p v)←⍵⋄(p+nv)(nv←v+Gravity p)}

In [280]:
⊢part1←+/Energy⌿↑ApplyForces⍣1000⊢DAY12 ((0 0 0)(0 0 0)(0 0 0)(0 0 0))
assert 8625=part1

8625

Nice.

Part 2 asks us to determine the number of steps that must occur before all of the moons' positions and velocities exactly match a previous point in time. There are strong hints that the number of iterations will likely be too large for us to brute. 

This is a trickier problem.

The key insight to exploit is that the column vectors for x, y and z are independent and periodic (or there would be no solution), so we search for the common period.

In [281]:
]dinput
Periods←{
    startState←↓⍉↑⍵                ⍝ zip to make col vectors
    periods←¯1 ¯1 ¯1               ⍝ zyx
    count←2
    _←{
        periods[⍸(¯1=periods)∧(↓⍉↑⊃0⌷⍵)≡⍤0⊢startState]←count
        ApplyForces ⍵
    }⍣{count+←1⋄~¯1∊periods}⊢ApplyForces ⍵ ((0 0 0)(0 0 0)(0 0 0)(0 0 0))
    periods
}

In [282]:
⊢periods←Periods DAY12

┌→──────────────────┐
│193052 96236 286332│
└~──────────────────┘

So that gives us the per-component periods, but what is the common period they must share? Fortunately for us, APL has lowest common multiple built in, which is handy.

In [285]:
⊢part2←∧/periods ⍝ wow apl :)
assert 332477126821644=part2

332477126821644

### Day 13: Care Package
https://adventofcode.com/2019/day/13

For that we now have to pay the mandated Intcode tax.

Part 1: run the program, count the number of output groups of three ending in a 2.

In [316]:
DAY13←⊢⌿⍎¨⎕CSV'data/2019/13.txt'

In [317]:
⊢part1←+/2=2⊃¨↓(3÷⍨≢out) 3⍴out←4⊃Step⍣{5⊃⍺}⊢(DAY13,512⍴0)0 0(,⍬)(,⍬)0 0
assert 193=part1

193

In [367]:
]dinput
Breakout←{
    code←⍵
    code[0]←2 ⍝ Enable unlimited quarters mode
    final←{
        state←Step⍣{(6⊃⍺)∨5⊃⍺}⊢⍵                       ⍝ Execute until blocked or halted
        5⊃state:state                                  ⍝ Game over
        buf←↑3 cut 4⊃state                             ⍝ Matrix from triplets
        data←⊢/buf                                  
        state[3],←buf[data⍳3;0](<->)buf[⊃¯1↑⍸4⍷data;0] ⍝ Paddle tracks last ball in x
        state[4]←⊂,⍬                                   ⍝ Reset output
        state
    }⍣{5⊃⍺}⊢(code,512⍴0)0 0(,0)(,⍬)0 0
    result←↑3 cut 4⊃final
    scoreidx←result[;0 1]⍳¯1 0
    result[scoreidx;2]
}

In [368]:
⊢part2←Breakout DAY13
assert 10547=part2

10547

### Day 14: Space Stoichiometry
https://adventofcode.com/2019/day/14

Solution as presented here heavily influenced by [voidhawk42](https://github.com/voidhawk42/aoc2019apl/blob/master/p14.dyalog). Python [version](https://github.com/xpqz/aoc-19/blob/master/day14.py).

In [481]:
'segs'⎕CY'dfns'
DAY14←↑{↓⍉↑⍵}¨{2 cut {6::⍵⋄⍎⍵}¨', =>' segs ⍵}¨lines'data/2019/14.txt'

In [482]:
TARGETS←(⊂'ORE'),⍨⊃,/¯1↑¨DAY14[;1]

Now we build a table of all the formulae, showing target amounts as positive values along the diagonal, and the ingredient 'costs' as negative weights.

In other words, if we have the rule

    (⊂1 1 2) ('HVXJL' 'JHGQ' 'ZQFQ')

we can expect the entry in the HVXJL and JHGQ cols to be ¯1 and that for the target ZQFQ to be 2.

Dyalog's [selective assigment](http://help.dyalog.com/18.0/#Language/Primitive%20Functions/Assignment%20Selective.htm).

In [484]:
FORMULAE←-↑{⍺@(TARGETS⍳⍵)⊢0⍴⍨≢TARGETS}/DAY14
(0 0⍉FORMULAE)×←¯1 ⍝ Flip sign along diagonal via selective assignment

In [490]:
]dinput
React←{ ⍝ React amnt
    ⊃-¯1↑{
        ∧/0≤¯1↓⍵:⍵                           ⍝ We're done. ORE is the last item.
        element←⊃⍸0>⍵                        ⍝ Pick first unfulfilled (negative) element
        items←element⌷FORMULAE               ⍝ ...and its corresponding ingredient weights
        ∇⍵+items×⌈(-element⌷⍵)÷element⌷items ⍝ Adjust remaining weights and add to queue
    } ⍵×FORMULAE⌷⍨TARGETS⍳⊂'FUEL'
}

In [489]:
⊢part1←React 1
assert 216477=part1

216477

Binary search

In [497]:
⊢part2←1 {⍺=⍵:⍵ ⋄ 1e12>React⊢m←⌈2÷⍨⍺+⍵:m∇⍵-1 ⋄ ⍺∇m} 2×⍣{1e12<React ⍺}⊢1
assert 11788286=part2

11788286

### Day 15: Oxygen System
https://adventofcode.com/2019/day/15

Guess what. Intcode AND graphy path findingy things.

We start with a breadth-first traversal of the maze to map it out. We can then apply Dijkstra's algorithm for parts 1 and 2. We could have tapped out the answer for part 1 directly from the BFS at the cost of some complexity.

In [49]:
DAY15←⊢⌿⍎¨⎕CSV'data/2019/15.txt'
ICState←{(⍵,512⍴0)0 0(,⍬)(,⍬)0 0}
Move←{ic←⍺ ⋄ ic[3],←⍵ ⋄ Step⍣{(6⊃⍺)∨5⊃⍺}⊢ic}

In [50]:
]dinput
N4←{
    (cur state)←⍵
    pos←(⊂cur)(-,+)(0 1)(1 0)                 ⍝ W N E S = 3 1 4 2
    unvisited←(≢⍺)=⍺⍳pos                         
    new←state∘Move¨3 1 4 2                    ⍝ Apply each move command (classify location)
    tiles←∊¯1↑¨(↑new)[;4]                     ⍝ Last item of each output buffer
    valid←unvisited∧0≠tiles                   ⍝ Unvisited non-wall positions
    (↓⍉↑(valid/pos)(valid/new))(pos/⍨2=tiles)
}

In [51]:
]dinput
YABFS←{ ⍝ Yet Another Breadth First Search
    spout←⍬
    ⍬ {
        0=≢⍵:⍺ spout
        head←0⊃⍵
        (newStates cand)←⍺ N4 head
        _←{0≠≢⍵:⊢spout⊢←⍵⋄⍬} cand
        (⍺,¯1↓head)∇(1↓⍵),newStates
    } ,⊂(100 100) (ICState ⍵) ⍝ Y X state-vec
}

In [54]:
(empties spout)←YABFS DAY15

Now we have a graph, and a target location, which should mean we're done with the Intcode bits, thankfully.

Looking back through the AoC archives, we've used Dijkstra's several times that we can use as a basis. This time, however -- and we should have done this ages ago -- let's have a dictionary. Come on, Dyalog, you know it makes sense!

In [42]:
]LINK.Create heapq src/heapq
]LINK.Create dict src/dict

┌→──────────────────────────────────────────────────────────────┐
│Linked: #.heapq → /Users/stefan/work/dyalog/AoCDyalog/src/heapq│
└───────────────────────────────────────────────────────────────┘

┌→────────────────────────────────────────────────────────────┐
│Linked: #.dict → /Users/stefan/work/dyalog/AoCDyalog/src/dict│
└─────────────────────────────────────────────────────────────┘

In [82]:
Neighbours←{⊂¨(⍸×⍺)∩⍵+(¯1 0)(1 0)(0 ¯1)(0 1)}
UnwindPath←{p←⍺⋄(s e)←⍵⋄⍬{⍵≡s:⊖⍺⋄(⍺,⍵)∇p ##.dict.Get ⍵}e}

In [100]:
]dinput
Dijkstra←{
    ⍝ Dijkstra's without cut-off
    ⍝ See https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm
    cost←⍵ ##.dict.Create 0 ⋄ cost.Default←⌊/⍬
    cameFrom←⍵ ##.dict.Create ⍵
    Neigh←⍺⍺
    Pick←{oldCost←cost ##.dict.Get ⍵⋄⍺<oldCost}                
    _←{
        (queue queueItem)←##.heapq.Pop ⍵
        current←1⊃queueItem
        newCost←1+cost ##.dict.Get current
        valid←(newCost∘Pick¨n)/n←Neigh current             ⍝ New, or better?
        _←cameFrom ##.dict.Set (valid) ((≢valid)⍴⊂current) ⍝ Update path dictionary
        _←cost ##.dict.Set (valid) ((≢valid)⍴newCost)      ⍝ Any changed costs
        queue ##.heapq.Push ↓⍉↑((≢valid)⍴newCost)(valid)   ⍝ Enqueue connected nodes
    }⍣{##.heapq.Empty ⍺} ##.heapq.Push ⊂0 ⍵
    cameFrom
}

In [101]:
graph←120 120⍴0
graph[empties]←1

In [102]:
cf←(graph∘Neighbours Dijkstra) ⊂100 100

In [103]:
⊢part1←≢cf UnwindPath (⊂100 100)(⊂88 80)
assert 248=part1

248

Part 2 can be expressed as finding the longest shortest path between the spout and any other open space. 

In [87]:
cf←(graph∘Neighbours Dijkstra) ⊂88 80

In [88]:
⊢part2←⌈/{≢cf UnwindPath (⊂⍺)(⊂⍵)}⌿↑((≢empties)⍴⊂88 80)(empties)
assert 382=part2

382

In retrospect, we could have used the Dijkstra routine directly, in place of the 'YABFS' function above, by having it record the location of the spout. But that would have made the Dijkstra rather fugly.

### Day 16: Flawed Frequency Transmission
https://adventofcode.com/2019/day/16

Ok, we're back on APL home-court advantage. It starts easy enough...

In [114]:
⎕io←1
DAY16←⍎¨line'data/2019/16.txt'

In [115]:
Coeff←{⍺↑1↓∊(1+⌈⍺÷4×⍵)⍴⊂∊⍵∘{⍺⍴⍵}¨0 1 0 ¯1}
Phase←{10||(↑(≢⍵)Coeff¨⍳≢⍵)+.×⍵}

In [116]:
⊢part1←8↑Phase⍣100⊢DAY16
assert 5 8 1 0 0 1 0 5≡part1

┌→──────────────┐
│5 8 1 0 0 1 0 5│
└~──────────────┘

In part 2, we're asked to repeat our input signal 10,000 times, making the naive approach intractable. However, as the coefficient matrix is upper-diagonal, we can skip a whole lot. Look at the coefficients for the example given:

In [117]:
{↑(≢⍵)Coeff¨⍳≢⍵}⍳8

┌→─────────────────┐
↓1 0 ¯1 0 1 0 ¯1 0│
│0 1 1 0 0 ¯1 ¯1 0│
│0 0 1 1 1 0 0 0│
│0 0 0 1 1 1 1 0│
│0 0 0 0 1 1 1 1│
│0 0 0 0 0 1 1 1│
│0 0 0 0 0 0 1 1│
│0 0 0 0 0 0 0 1│
└~─────────────────┘

The first 7 digits of our input now represents the offset to the sub-sequence we're interested in. Given the zeroes in the coefficient matrix, only the sub-sequence actually contributes, so can skip everything before. Each digit depends on the previous (digit n depends on digit n-1), as the zeros are followed by ones.

In [139]:
⊢offset←10⊥7↑DAY16

5970443

In [141]:
⊢part2←8↑{1↓⊖10|+\(⊖⍵),0}⍣100⊢offset↓∊10000⍴⊂DAY16
assert 4 1 7 8 1 2 8 7≡part2

┌→──────────────┐
│4 1 7 8 1 2 8 7│
└~──────────────┘

That was nice, wasn't it? I learnt something after trying first to do:

    {1↓⊖{10|⍺+⍵}\(⊖⍵),0}
    
which is crushingly slow. Scan with a dfn operand ends up being O(n^2), where as +\ hits a highly efficient optimised idiom. 

### Day 17: Set and Forget
https://adventofcode.com/2019/day/17

Anyway. Let's put that lovely problem aside, and get back to some Intcode :/

In [295]:
⎕IO←0
DAY17←⊢⌿⍎¨⎕CSV'data/2019/17.txt'

In [296]:
]dinput
MakeGraph←{
    graph←↑10(≠⊆⊢)4⊃Step⍣{(6⊃⍺)∨5⊃⍺}⊢(⍵,10000⍴0)0 0(,⍬)(,⍬)0 0
    dir←⊃(∪∊graph)~46 35
    (1@(⍸35=graph)⊢(⍴graph)⍴0)(⊃⍸dir=graph)dir
}

In [297]:
(g loc hd)←MakeGraph DAY17

In [298]:
cross←3 3 ⍴ 0 1 0 1 1 1 0 1 0   ⍝ All intersections look like a 3x3 'cross'
⊢part1←+/×/↑⍸({cross≡⍵}⌺3 3) g
assert 8408=part1

8408

Part 2 is considerably more complex. We need to trace the path given in the graph as a set of movement instructions, and partition it into a set of chunks no longer than 20 characters. We then execute these chunks on the Intcode computer, and -- if successful -- it will return a large number which is our answer.

To make the path, we do the following:

1. Follow straight line for as long as possible.
2. Go straight through any crossing.

In [314]:
Tuples←⊢⊂⍨≢⍤⊢⍴⊣↑≢
Merge←{⍵⊂⍨1,2≠/⍵}

In [315]:
]dinput
MakePath←{
    ⍺←⍬ ⋄ (graph pos dir)←⍵
    N←(⊂pos)(-,+)(1 0)(0 1)
    graph∊⍨⊂(⊃dir)⊃N:(⍺,1)∇graph ((⊃dir)⊃N) dir              ⍝ Straight on
    graph∊⍨⊂(⊃1⊖dir)⊃N:(⍺,'L')∇graph ((⊃1⊖dir)⊃N) (1⊖dir)    ⍝ Left
    graph∊⍨⊂(⊃¯1⊖dir)⊃N:(⍺,'R')∇graph ((⊃¯1⊖dir)⊃N) (¯1⊖dir) ⍝ Right
    2 Tuples ∊{(≢⍵)>1:1+≢⍵⋄⍵}¨Merge ⍺
}

In [316]:
path←MakePath (⍸g) loc (0 1 2 3)

So now we need to partition this into three sets of repeating sequences (A, B and C), each less than or equal to 20 characters in length, including encoding commas. 

1. Start from the beginning with gradually longer sequences, making the A sequence.
2. Remove all A matches, and try the same, now making gradually longer B sequences.
3. Remove all B matches, now making gradually longer C sequences.
4. If when we remove all C-matches we have an empty vector, we've found a solution.
5. Otherwise, repeat from 1, increasing the length of A's pattern by 1 instruction

We assert that each sequence starts with a turn -- this may or may not be true in the general case, but works for the given input set. There may well be multiple possible solutions; we're picking the first one we find.

We can pre-generate all likely length combinations: longer than 5 is likely breaking the max encoding length.

In [317]:
Enc←{¯1↓'\s'⎕R''⊢∊{⍺,',',⍕⍵,','}/↑⍵}
Test←{rm←∊(patt(⍸⍷)⍵)+⊂⍳≢patt←⊃⍺ Tuples ⍵⋄(patt)(⍵/⍨0@rm⊢1⍨¨⍵)}

In [318]:
]dinput
Partition←{
    ⍵ {
        (a b c)←⊃⍵
        (apatt apath)←a Test ⍺
        (bpatt bpath)←b Test apath
        (cpatt cpath)←c Test bpath
        (0=≢cpath)∧∧/20>≢∘Enc¨apatt bpatt cpatt:apatt bpatt cpatt 
        ⍺∇1↓⍵
    } ,1+⍳5 5 5
}

In [319]:
Partition path

┌→────────────────────────────────────────────────────────────────────────────────────┐
│ ┌→─────────────────────────┐ ┌→────────────────────┐ ┌→───────────────────────────┐ │
│ │ ┌→──┐ ┌→───┐ ┌→──┐ ┌→──┐ │ │ ┌→───┐ ┌→──┐ ┌→───┐ │ │ ┌→───┐ ┌→───┐ ┌→──┐ ┌→───┐ │ │
│ │ │R 6│ │L 10│ │R 8│ │R 8│ │ │ │R 12│ │L 8│ │L 10│ │ │ │R 12│ │L 10│ │R 6│ │L 10│ │ │
│ │ └+──┘ └+───┘ └+──┘ └+──┘ │ │ └+───┘ └+──┘ └+───┘ │ │ └+───┘ └+───┘ └+──┘ └+───┘ │ │
│ └∊─────────────────────────┘ └∊────────────────────┘ └∊───────────────────────────┘ │
└∊────────────────────────────────────────────────────────────────────────────────────┘

So now we need to dive back into Intcode domain again. First we turn the partitioned path into input suitable for the Intcode program, as specified.

In [320]:
]dinput
Compile←{
    (a b c)←Partition ⍵
    p←⍉↑(∊a b c(⍸⍷)¨⊂path)(¯1↓∊a b c{⍵⍴⍨≢⍺}¨'ABC') ⍝ Line up start indices and letters
    main←1⊃↓⍉p[⍋p[;0];] ⍝ Sort by start index
    m←⎕UCS','(1↓∘,,⍤0) main
    (A B C)←⎕UCS∘Enc¨a b c
    m,10,A,10,B,10,C,10,110,10
}

In [321]:
instr←Compile path

We're getting there. Now we need to do three things:
1. Enter the magic number 2 in position 0 of the Intcode program, which wakes the robot.
2. Give the Intcode program the set of movement instructions we just compiled.
3. Run the robot to completion, and collect its final output.

In [322]:
DAY17[0]←2
⊢part2←⊃¯1↑4⊃Step⍣{(6⊃⍺)∨5⊃⍺}⊢(DAY17,10000⍴0)0 0(instr)(,⍬)0 0
assert 1168948=part2

1168948

### Day 18: Many-Worlds Interpretation
https://adventofcode.com/2019/day/18

Ok, now it starts to bite, and it's only day 18! Back to path finding in a colossal cave -- an AoC stalwart problem style. This is an interesting, but difficult problem.

We need to visit a certain number of points in a maze, following the shortest possible path. The complication is that there are doors that need unlocking. The approach taken here is to pre-calculate the shortest path between every key pair, and recording all doors that would need passing, and any keys picked up along the way.

Each such path can then be seen as a vertex pair connected by an edge whose weight is the path length, and the doors along the path indicates the keys that need to be held in orther to traverse it.

We can (more or less) use the same path finding code to find the optimal order to visit each key location as that which we used to find the shortest path between key pairs.

In [5]:
DAY18←↑lines'data/2019/18.txt'

In [6]:
LC←¯1⎕C⎕A   ⍝ Keys are a subset of lower case letters

In [7]:
keys←(LC∊DAY18)/LC
keylen←1+≢keys
GRAPH←'#'≠DAY18
KEYS←DAY18∘{⊃⍸⍵⍷⍺}¨'@',keys

In [73]:
]dinput
Dijkstra←{
    ⍝ Dijkstra's WITH cut-off
    ⍝ See https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm
    target←⍵
    cost←⍺ ##.dict.Create 0 ⋄ cost.Default←⌊/⍬
    cameFrom←⍺ ##.dict.Create ⍺
    Neigh←⍺⍺
    Pick←{oldCost←cost ##.dict.Get ⊂⍵⋄⍺<oldCost}                
    _←{
        (queue queueItem)←##.heapq.Pop ⍵
        current←1⊃queueItem
        target≡current:⍬
        newCost←1+cost ##.dict.Get current
        valid←(newCost∘Pick¨n)/n←Neigh current                   ⍝ New, or better?
        _←cameFrom ##.dict.Set (valid) ((≢valid)⍴current)        ⍝ Update path dictionary
        _←cost ##.dict.Set (valid) (validCosts←(≢valid)⍴newCost) ⍝ Any changed costs
        queue ##.heapq.Push ↓⍉↑validCosts (⊂¨valid)              ⍝ Enqueue connected nodes
    }⍣{##.heapq.Empty ⍺} ##.heapq.Push ⊂0 ⍺
    cameFrom
}

In [9]:
Neighbours←{(⍸×⍺)∩⍵+(¯1 0)(1 0)(0 ¯1)(0 1)}
UnwindPath←{p←⍺⋄(s e)←⍵⋄⍬{⍵≡s:⊖⍺⋄(⍺,⍵)∇p ##.dict.Get ⍵}e}

We want to pre-calculate the shortest route between every key-pair, recording any doors and keys along the path.

In [10]:
]dinput
KeyDistances←{
    (a b)←⍵
    a=b:⍬
    cf←(a⌷KEYS)(GRAPH∘Neighbours Dijkstra)(b⌷KEYS) ⍝ Shortest path
    (a⌷KEYS),cf UnwindPath (a⌷KEYS) (b⌷KEYS)       ⍝ Add start key to the path
}

In [11]:
KeysAndDoors←{⍬≡⍵:⍬ ⋄ ⍺[⍵]~'.#@'}           ⍝ Extract doors and keys along a path

In [12]:
kd←KeyDistances¨,(⊢×∘.≤⍨∘⍳∘≢)⍳keylen keylen ⍝ Upper-triangular only; still slow
kad←keylen keylen⍴DAY18∘KeysAndDoors¨kd
KEYSDOORS←,⌿↑kad (⊖¨⍉kad)                   ⍝ Reflect and reverse upper triangular
KEYSDOORS[;0]←keylen ⍴⊂⍬                    ⍝ Wipe returns to start
pd←keylen keylen ⍴≢¨kd
PATHLEN←+⌿↑pd(⍉pd)

KEYSDOORS defines the doors and keys collected on the shortest path between any pair of keys. We want to visit every key following the shortest cumulative path possible, given the constraint that in order to traverse a door, we need to hold the corresponding key.

So what we have now we can think of as a graph where each node is a key, and each key connected to a set of 0 or more other keys, depending on the 'keyring' carried at each point. Each connection has a cost (the length of the path) and travelling may or may not add to the 'keyring'. Our Dijkstra routine above assumes a uniform edge cost of 1, which won't work here.

We define a node in our graph as the key and the set of keys held in order to get there. Note to the wary: this was hard to get right.

In [13]:
]dinput
Reachable←{
    ⍝ Return costed connected nodes from key, given path (the held keys) and 
    ⍝ a current cost dist.
    (key path dist)←⍵
    doorID←{⍵≡'':0 ⋄ 1+LC⍳⍵}key

    keys←⍬ {
        0=≢⍵:∪⍺
        k←⊃((⊃⍵)~(1 ⎕C path))~path
        k∊LC:(⍺,k)∇ 1↓⍵
        ⍺ ∇ 1↓⍵
    } doorID⌷KEYSDOORS

    ⍝ We also need the accumulative costs
    ids←1+LC⍳⊢/↑conn←(⊂path),¨keys
    ↓⍉↑(keys)(conn)(¯1+dist+PATHLEN[doorID,¨ids]) ⍝ ¯1 to not double-count end-becomes-start node
}

In [19]:
]dinput
FindPath←{
    ⍝ Another spin on the Dijkstra theme.
    ⍝  - cutoff
    ⍝  - non-uniform edge cost
    ⍝  - complex nodes
    ⍝
    ⍝ Complication: a node is a key + the set of keys required to be there (regardless of path)
    ⍝ However, the path order matters for the Dijkstra alg itself, but we need to be order-independent
    ⍝ when we check if a node has been visited before.
    start←⍵ ⋄ found←⍬
    Vertex←{(0⊃⍵)(sorted 1⊃⍵)}
    cost←(⊂Vertex⊃start)##.dict.Create 0 ⋄ cost.Default←⌊/⍬
    seen←⍬
    Neigh←⍺⍺ ⋄ Exit←⍵⍵
    _←{
        (queue queueItem)←##.heapq.Pop ⍵
        current←⊃1⊃queueItem
        node←Vertex current
        (cost ##.dict.Get⊂node)<2⊃current:queue
        Exit 1⊃current:⍬⊣found⊢←current
        _←{(⊂⍵)∊seen:⍬ ⋄ ⊢seen,←⊂⍵}node
        connected←Neigh current
        0=≢connected:queue
        nodes←connected/⍨~(Vertex¨connected)∊seen

        newCosts←⊢/↑nodes

        valid←>⌿↑({cost ##.dict.Get⊂⍵}¨Vertex¨nodes)(newCosts)

        0=∨/valid:queue
        validNodes←valid/nodes
        validCosts←valid/newCosts

        _←cost ##.dict.Set(Vertex¨validNodes)(validCosts)
        queue ##.heapq.Push↓⍉↑(validCosts)(⊂¨validNodes)
    }⍣{##.heapq.Empty ⍺}##.heapq.Push⊂0 start
    found
}

In [21]:
⊢part1←(Reachable FindPath {''≡keys~⍵}) ⊂''(,⍬)0
assert 4406=2⊃part1

┌→────────────────────────────────────┐
│ ┌→─────────────────────────┐ │
│ l │svdunpmagifwehjqotkyrzcbxl│ 4406 │
│ - └──────────────────────────┘ │
└∊────────────────────────────────────┘

Part 2: now we have 4 robots, and a partitioned maze to deal with. This is pretty much the same, but the state at each point is now 4 keys (one per quadrant), and we need to constrain the Reachable function to only look within the same quadrant as the current key.

In [27]:
middle←↓⍉↑¯1 0 1+⊂⌊2÷⍨⍴DAY18
DAY18[(0⊃middle);(1⊃middle)]←3 3⍴'@#@###@#@' ⍝ Structural change: first moves will be two steps shorter!   
Q←{2⊥~⍵}¨(1↓KEYS)< ⊂⌊2÷⍨⍴DAY18               ⍝ Quadrant of keys, 2 2⍴⍳4
ALLKEYS←keys

In [33]:
]dinput
Reachable2←{ ⍝ also constrain by quadrant
    (qkey path dist)←⍵
    ⊃,/{
        quadrant←⍵
        doorID←{⍵≡'@':0⋄1+LC⍳⍵}⍵⊃qkey
        neighs←⍬{
            0=≢⍵:∪⍺
            k←⊃((⊃⍵)~(1⎕Cpath))~path
            (k=0)∨~k∊LC:⍺∇1↓⍵
            quadrant=Q[ALLKEYS⍳k]:(⍺,k)∇1↓⍵
            ⍺∇1↓⍵
        }doorID⌷KEYSDOORS
        ids←1+LC⍳⊢/↑newKeyRings←(⊂path),¨neighs
        ↓⍉↑({⍵@quadrant⊢qkey}¨neighs)(newKeyRings)(¯1+dist+PATHLEN[doorID,¨ids])
     }¨⍳4
 }

In [34]:
⊢part2←(Reachable2 FindPath {''≡keys~⍵}) ⊂'@@@@'(,⍬)0
assert 1964=¯8+2⊃part2 ⍝ Subtract 8, first move is 2 shorter for each robot due to structural change

┌→─────────────────────────────────────────┐
│ ┌→───┐ ┌→─────────────────────────┐ │
│ │hxyl│ │unpamsvdwjqgitkyrefhozcbxl│ 1972 │
│ └────┘ └──────────────────────────┘ │
└∊─────────────────────────────────────────┘

### Day 19: Tractor Beam
https://adventofcode.com/2019/day/19

O yeah, Intcodez.

In [36]:
DAY19←⊢⌿⍎¨⎕CSV'data/2019/19.txt'
Beam←{⊃⊃4⊃Step⍣{(6⊃⍺)∨5⊃⍺}⊢(DAY19,512⍴0) 0 0(⍵,⍺)(,⍬)0 0} ⍝ Step from Day09

In [37]:
⊢part1←+/+/Beam/¨⍳50 50
assert 169=part1

169

Part 2 -- find the location where a 100×100 square will have its off-diagonal corners touching the sides of the tractor beam. We know that the leading zeros of a given line will be increasing by at least 1 from the previous, so we can skip a load of tests. So: take the first 1 on a row as a candidate lower-left corner of the square. Check the upper-right. If this is zero, move one row down.

In [38]:
]dinput
FitSquare←{
    (x dx)←⍵ ⋄ p←+/⍵               ⍝ ⍺ is the current row (y)
    ~⍺ Beam p:⍺∇x,dx+1             ⍝ Lower-left
    (⍺-99) Beam p+99:¯99+⍺+10000×p ⍝ Upper-right
    (⍺+1)∇p,0 
}

In [40]:
⊢part2←100 FitSquare 0 0
assert 7001134=part2

7001134

### Day 20: Donut Maze
https://adventofcode.com/2019/day/20

Moar Dijkstras! Should be able to re-use the Dijkstra-routine from Day18.

In [85]:
DAY20←↑lines'data/2019/20.txt'
FLOOR←⍸'.'=DAY20
UnwindPath←{p←⍺⋄(s e)←⍵⋄⍬{⍵≡s:⊖⍺⋄(⍺,⍵)∇p ##.dict.Get ⍵}e}

In [86]:
Doors←{(d x)←{(↓⍉⍵[;⍸cols])(⍸cols←(¯1 ¯1)≢⍤1⊢⎕A⍸⍉⍵)} (⍺⍺ DAY20)[⍵;]⋄(d)(↓⍺,[.5]x)}

Portals are either on the outside "surface" or the inside. We can cheat a bit by locating those rows manually. We can reuse our door extraction logic for both horizontal and vertical doors by transposing the matrix.

In [87]:
]dinput
DoorLocations←{
    horiz←↑,/⍉↑(⍉Doors)⌿↑(2 26 80 104)((0 1) (27 28) (78 79) (105 106))
    horiz[1;]←⊖¨horiz[1;]
    vert←↑,/⍉↑(⊢Doors)⌿↑(2 26 82 106)((0 1) (27 28) (80 81) (107 108))
    all←horiz,vert
    tmp←⍉all[;⍋all[0;]]
    (⊃(1↑tmp)[0;1]) (⊃(¯1↑tmp)[0;1]) (1⌷⍉¯1↓1↓tmp) ⍝ entry, exit and doors. 
}

In [88]:
(AA ZZ DOORS)←DoorLocations⍬

In [91]:
]dinput
N4←{ 
    n←⍺∩⍵(+,-)(0 1)(1 0)             ⍝ 4-connected neighbours
    (≢DOORS)=idx←DOORS⍳⍵: n
    n,((1+idx)(¯1+idx)[2|idx])⌷DOORS ⍝ Portal. Even index map to +1, odd to -1
}

In [92]:
cf←(⊂AA) (FLOOR∘N4 Dijkstra) ⊂ZZ

In [93]:
⊢part1←≢cf UnwindPath (⊂AA)(⊂ZZ)
assert 400=part1

400